##1. Get authenticated from tweeter API

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/NLP'

apikey= 'du9vEcqQFCtdu9vEcqQFCt'
apisecretkey= 'w7JsH0E4KCfOTMdcEw7JsH0E4KCfOTMdcE'
accesstoken='126864770-A5bLHXKcDFFk126864770-A5bLHXKcDFFk'
accesstokensecret='JS1l5hXn9dSM8lfJS1l5hXn9dSM8lf'

import os
import tweepy as tw

auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)

api = tw.API(auth,wait_on_rate_limit=True)

In [ ]:
dir(api)

#2: Know what's on my timeline

In [ ]:
my_timeline = api.home_timeline()
for status in my_timeline:
  #print(status.text)
  print(vars(status))

In [ ]:
columns = set()
allowed_types = [str,int]
tweets_data = []
for status in my_timeline:
  status_dict = dict(vars(status))
  keys = vars(status).keys()
  single_tweet_data = {'user':status.user.screen_name, 'author':status.author.screen_name}
  for k in keys:
    try:
      #print(k)
      v_type=type(status_dict[k])
    except:
      v_type = None
    if v_type != None:

        if v_type in allowed_types:
          single_tweet_data[k] = status_dict[k]
          columns.add(k)

  tweets_data.append(single_tweet_data)
header_cols = list(columns)

header_cols.append('user')
header_cols.append('author')
header_cols

In [ ]:
tweets_data

In [ ]:
import pandas as pd
df = pd.DataFrame(tweets_data,columns = header_cols)
df.head()

#3: What's on different user's timeline

In [ ]:
def extract_timeline_as_df(timeline_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in timeline_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"user": status.user.screen_name, "author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)


    header_cols = list(columns)
    header_cols.append("user")
    header_cols.append('author')
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [ ]:
my_timeline = api.home_timeline()

In [ ]:
df2 = extract_timeline_as_df(my_timeline)
df2.head()

In [ ]:
user = api.get_user("sudhirchaudhary")
user_timeline = user.timeline()
df3 = extract_timeline_as_df(user_timeline)
df3.head()

#4: Reply any tweet 

In [ ]:
# og_tweet = api.get_status("1584907651753746432")
# # print(og_tweet.user.screen_name, og_tweet.id)
# my_reply = api.update_status(f"@{og_tweet.user.screen_name} Wow this cool!", og_tweet.id)
#print(my_reply.id, my_reply.user.screen_name)

#5: Find all followers and friends count

In [ ]:
user = api.get_user("ipvikas")
print(user.followers_count, user.friends_count)

In [ ]:
user_friends = user.friends()
for friend in user_friends:
  print(friend.screen_name)

#6: Items & Pagination with Tweepy Cursor

In [ ]:
len(api.home_timeline(count=20))

In [ ]:
import tweepy

user = api.get_user("sudhirchaudhary")

for i, status in enumerate(tweepy.Cursor(api.home_timeline, count=50).items(50)):
    print(i, status.text)

#7: Search for any tweet from any start date

In [ ]:
search_words = ["#SuryaGrahan"] # ["#ipl2020","#DCvKXIP"]; ["#coronavirus","#Modi"]; ["#TwinTowers"] 
date_since = "2022-10-25"

In [ ]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(10)

tweets

#will take some time 2 min
# tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location,tweet.created_at,[e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count] for tweet in tweets]
tweet_details

In [ ]:
import pandas as pd
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user', "location","Timestamp","All_hashtags","Followers_Count"])
pd.set_option('max_colwidth', 80)
tweet_df.head(5)

In [ ]:
tweet_df.user.value_counts()

In [ ]:
tweet_df.location.value_counts()

In [ ]:

# emoticons_happy = set([
#     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#     '<3'
#     ])
 

# emoticons_sad = set([
#     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#     ':c', ':{', '>:\\', ';('
#     ])
 

# emoticons = emoticons_happy.union(emoticons_sad)

# import re
# def clean_tweets(text):
#     text = re.sub("RT @[\w]*:","",text)
#     text = re.sub("@[\w]*","",text)
#     text = re.sub("https?://[A-Za-z0-9./]*","",text)
#     text = re.sub("\n","",text)


#     text = re.sub("\$\w*","",text)
#     text = re.sub("^RT[\s]+","",text)
#     text = re.sub("https?:\/\/.*[\r\n]*","",text)
#     text = re.sub("#","",text)


#     tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#     tweet_tokens = tokenizer.tokenize(text)
 
#     tweets_clean = []    
#     for word in tweet_tokens:
#         if (word not in stopwords_english and
#               word not in emoticons and 
#                 word not in string.punctuation): 
           
#             stem_word = stemmer.stem(word)
#             tweets_clean.append(stem_word)
 
#     return tweets_clean
#     text = tweets_clean 
 



#     # return text

# tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

# tweet_df.head(20)
# tweet_df.to_csv('tweets.csv')
# # !ls



# # tweet = re.sub(r'\$\w*', '', tweet)
# # tweet = re.sub(r'^RT[\s]+', '', tweet)
# # tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
# # tweet = re.sub(r'#', '', tweet)

In [ ]:
import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)


    text = re.sub("\$\w*","",text)
    text = re.sub("^RT[\s]+","",text)
    text = re.sub("https?:\/\/.*[\r\n]*","",text)
    text = re.sub("#","",text)

    return text

tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

tweet_df.head(20)
tweet_df.to_csv('tweets.csv')
# !ls

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else ""  for ent in nlp(x).ents])

In [ ]:
tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])
tweet_df.head(20)

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [ ]:
tweet_df.head(10)

In [ ]:
!pip install googlemaps

In [ ]:
import googlemaps

In [ ]:
def get_country(input):
  try:
    output=gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error"
  return output

tweet_df['country']=tweet_df['location'].apply(lambda x: "" if (not x.strip()) else get_country(x))

tweet_df['country'].value_counts()

In [ ]:
tweet_df.head(10)

In [ ]:
tweet_df.dtypes

In [ ]:
tweet_df.to_csv("tweets.csv")

In [ ]:
#BASIC 1
import csv
import random
from nltk.corpus import twitter_samples as ts
import tweepy
#in anaconda prompt: pip install tweepy
from textblob import TextBlob

consumer_key= 'du9vEcqQFCtidu9vEcqQFCti'
consumer_secret= 'w7JsH0E4KCfOTMdcw7JsH0E4KCfOTMdc'

access_token='126864770-A126864770-A'
access_token_secret='JS1l5hXn9JS1l5hXn9'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

#Step 3 - Retrieve Tweets
public_tweets = api.search('SuryaGrahan')

for tweet in public_tweets:
    print(tweet.text)
    
    #Step 4 Perform Sentiment Analysis on Tweets
    analysis = TextBlob(tweet.text)
    print(analysis.sentiment)
    print("")

In [ ]:
#BASIC 2: we have used it in above code
# https://github.com/ritvikmath/ScrapingData
import json
import csv
import tweepy
#in anaconda prompt: pip install tweepy

import re

"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""

def search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase):
    #create authentication for accessing Twitter
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    #initialize Tweepy API
    api = tweepy.API(auth)
    
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", hashtag_phrase))

    #open the spreadsheet we will write to
    with open('%s 2_tweets.csv' % (fname), 'w') as file:
        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['timestamp', 'tweet_text', 'username', 'all_hashtags', 'followers_count'])

        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        for tweet in tweepy.Cursor(api.search, q=hashtag_phrase+' -filter:retweets', lang="en", tweet_mode='extended').items(100):
            w.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8'), [e['text'] for e in tweet._json['entities']['hashtags']], tweet.user.followers_count])
            
# consumer_key = input('Consumer Key')
# consumer_secret = input('Consumer Secret')
# access_token = input('Access Token')
# access_token_secret = input('Access Token Secret')
consumer_key= 'du9vEcqQFCtiiNSJYpDhjJwEW'
consumer_secret= 'w7JsH0E4KCfOTMdcEtSKa3RSo8CImQVlDW6cSeujpBT2mKTigS'
access_token='126864770-A5bLHXKcDFFkLqxFG296m56xlvGzskw4WMTIfRq5'
access_token_secret='JS1l5hXn9dSM8lf2TCsX6g8JpLzFA5DyQ2znHGvgNr1rD'

hashtag_phrase = input('Hashtag Phrase')

if __name__ == '__main__':
    search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)

## 2. Scrape Twitter Data or Tweets using snscrape module

In [ ]:
!pip install -q snscrape

import os
import pandas as pd
from datetime import date, timedelta

In [ ]:
today = date.today()
end_date = today

search_term = 'SuryaGrahan'
from_date = date.today() - timedelta(days=1)# '2022-08-28'

####Total Number of Tweets for Search Terms

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/My Drive/Colab Notebooks/NLP'

In [ ]:
import os
# from_date = '2021-08-15'
# search_term = '#IndiaIndependenceDay'
# end_date = '2021-08-16'

os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt") #will take more than 10 minutes
if os.stat("result-tweets.txt").st_size == 0:
  counter = 0
else:
  df = pd.read_csv('result-tweets.txt', names=['link'])
  counter = df.size

print('Number Of Tweets : '+ str(counter))

#####Extracting Exact Tweeets

In [ ]:
max_results = 100

In [ ]:
extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term} until:{end_date}' > extracted-tweets.txt"

# extracted_tweets = (re.sub('@[a-z,_,A-Z,0-9,:]+ ', '', i) for i in extracted_tweets)

os.system(extracted_tweets)
if os.stat("extracted-tweets.txt").st_size == 0:
  print('No Tweets found')
else:
  df = pd.read_csv('extracted-tweets.txt', names=['content'])
  for row in df['content'].iteritems():
    print(row)